In [1]:
import numpy as np
import pandas as pd

In [18]:
data = pd.read_csv('../data/processed/trials.csv')
data.drop(columns='nct_id', inplace=True)

In [19]:
for col in data.columns:
    if data[col].dtype not in [np.float64, np.int64]:
        print(col, ":", data[col].unique(), ":", data[col].dtype, "\n")

biospec_retention : ['UNKNOWN' 'SAMPLES_WITH_DNA' 'SAMPLES_WITHOUT_DNA' 'NONE_RETAINED'] : object 

is_ppsd : [False  True] : bool 

phase : ['NONE' 'PHASE2' 'PHASE4' 'PHASE2/PHASE3' 'PHASE1' 'PHASE3' 'EARLY_PHASE1'
 'PHASE1/PHASE2'] : object 

is_unapproved_device : [False  True] : bool 

is_fda_regulated_device : ['f' 'u' 't'] : object 

is_fda_regulated_drug : ['f' 'u' 't'] : object 

has_us_facility : ['f' 't' 'u'] : object 

has_max_age : [ True False] : bool 

has_min_age : [ True False] : bool 

primary_purpose : ['UNKNOWN' 'TREATMENT' 'HEALTH_SERVICES_RESEARCH' 'SUPPORTIVE_CARE'
 'BASIC_SCIENCE' 'PREVENTION' 'DIAGNOSTIC' 'OTHER' 'SCREENING' 'ECT'
 'DEVICE_FEASIBILITY'] : object 

model : ['COHORT' 'CASE_ONLY' 'PARALLEL' 'SINGLE_GROUP' 'OTHER' 'CROSSOVER'
 'SEQUENTIAL' 'UNKNOWN' 'CASE_CONTROL' 'FACTORIAL' 'FAMILY_BASED'
 'ECOLOGIC_OR_COMMUNITY' 'CASE_CROSSOVER' 'DEFINED_POPULATION'
 'NATURAL_HISTORY'] : object 

agency_class : ['OTHER' 'OTHER_GOV' 'INDUSTRY' 'FED' 'NIH' 'NETWORK

In [23]:
from sklearn.model_selection import train_test_split
X = data.drop(columns='terminated')
y = data['terminated']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [24]:
categoricals = [col for col in data.columns if data[col].dtype == np.object_] 
categoricals

['biospec_retention',
 'phase',
 'is_fda_regulated_device',
 'is_fda_regulated_drug',
 'has_us_facility',
 'primary_purpose',
 'model',
 'agency_class']

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [29]:
ohe = OneHotEncoder()
ohe.fit_transform(X_train[categoricals])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2032320 stored elements and shape (254040, 56)>

In [30]:
ohe.categories_

[array(['NONE_RETAINED', 'SAMPLES_WITHOUT_DNA', 'SAMPLES_WITH_DNA',
        'UNKNOWN'], dtype=object),
 array(['EARLY_PHASE1', 'NONE', 'PHASE1', 'PHASE1/PHASE2', 'PHASE2',
        'PHASE2/PHASE3', 'PHASE3', 'PHASE4'], dtype=object),
 array(['f', 't', 'u'], dtype=object),
 array(['f', 't', 'u'], dtype=object),
 array(['f', 't', 'u'], dtype=object),
 array(['BASIC_SCIENCE', 'DEVICE_FEASIBILITY', 'DIAGNOSTIC', 'ECT',
        'HEALTH_SERVICES_RESEARCH', 'OTHER', 'PREVENTION', 'SCREENING',
        'SUPPORTIVE_CARE', 'TREATMENT', 'UNKNOWN'], dtype=object),
 array(['CASE_CONTROL', 'CASE_CROSSOVER', 'CASE_ONLY', 'COHORT',
        'CROSSOVER', 'DEFINED_POPULATION', 'ECOLOGIC_OR_COMMUNITY',
        'FACTORIAL', 'FAMILY_BASED', 'NATURAL_HISTORY', 'OTHER',
        'PARALLEL', 'SEQUENTIAL', 'SINGLE_GROUP', 'UNKNOWN'], dtype=object),
 array(['AMBIG', 'FED', 'INDIV', 'INDUSTRY', 'NETWORK', 'NIH', 'OTHER',
        'OTHER_GOV', 'UNKNOWN'], dtype=object)]

In [ ]:
from sklearn.compose import ColumnTransformer